# Packed ensemble submission process

## Imports

In [1]:
import numpy as np
import pandas as pd
import os
import pickle
import torch
import torch.nn.functional as F

from lips import get_root_path
from lips.dataset.scaler.standard_scaler import StandardScaler
from lips.benchmark.airfransBenchmark import AirfRANSBenchmark
from lips.dataset.airfransDataSet import download_data
from lips.augmented_simulators.torch_simulator import TorchSimulator
from lips.dataset.scaler.standard_scaler_iterative import StandardScalerIterative

#from my_custom_packed_ensemble import *
#from my_packed_cv import *
from my_augmented_simulator import *

## Generic Step (Load the required data) <a id='generic_step'></a>

In [2]:
# indicate required paths
LIPS_PATH = get_root_path()
DIRECTORY_NAME = '../ml4physim_startingkit/Dataset'
BENCHMARK_NAME = "Case1"
LOG_PATH = LIPS_PATH + "lips_logs.log"

Define the configuration files path, that aim to describe specific caracteristics of the use case or the augmented simulator.

In [3]:
BENCH_CONFIG_PATH = os.path.join("airfoilConfigurations", "benchmarks",
                                 "confAirfoil.ini")  #Configuration file related to the benchmark
SIM_CONFIG_PATH = r"config.ini"  #Configuration file re

Download the data

In [4]:
if not os.path.isdir(DIRECTORY_NAME):
    download_data(root_path=".", directory_name=DIRECTORY_NAME)

Loading the dataset using the dedicated class used by LIPS platform offers a list of advantages:

1. Ease the importing of datasets
1. A set of functions to organize the `inputs` and `outputs` required by augmented simulators


In [5]:
# Load the required benchmark datasets
try:
    with open('benchmark.pkl', 'rb') as f:
        benchmark = pickle.load(f)
except:
    benchmark = AirfRANSBenchmark(benchmark_path=DIRECTORY_NAME,
                                  config_path=BENCH_CONFIG_PATH,
                                  benchmark_name=BENCHMARK_NAME,
                                  log_path=LOG_PATH)
    benchmark.load(path=DIRECTORY_NAME)
    with open('benchmark.pkl', 'wb') as f:
        pickle.dump(benchmark, f)

## Simulation

### Model creation

In [6]:
chunk_sizes=benchmark.train_dataset.get_simulations_sizes()
no_norm_x=benchmark.train_dataset.get_no_normalization_axis_indices()
scalerParams={"chunk_sizes":chunk_sizes,"no_norm_x":no_norm_x}

torch_sim = TorchSimulator(name="packed_mlp",
                           model=PackedMLP,
                           scaler=StandardScalerIterative,
                           scalerParams=scalerParams,
                           log_path=None,
                           device="cuda:0",
                           seed=42,
                           bench_config_path=BENCH_CONFIG_PATH,
                           bench_config_name="Benchmark1",
                           sim_config_path=SIM_CONFIG_PATH,
                           sim_config_name="DEFAULT",
                          )

### Training

In [8]:
torch_sim.train(benchmark.train_dataset, 
                save_path=None, 
                epochs=3, 
                train_batch_size=128000,
                pin_memory=True, 
                non_blocking=True, 
                num_workers=6
                )

Train Epoch: 0   Avg_Loss: 0.32531 ['MAELoss: 0.28161']
Train Epoch: 1   Avg_Loss: 0.17452 ['MAELoss: 0.19592']
Train Epoch: 2   Avg_Loss: 0.15410 ['MAELoss: 0.18411']


### Metric evaluation

In [10]:
torch_sim_metrics = benchmark.evaluate_simulator(augmented_simulator=torch_sim,
                                                  eval_batch_size=256000,
                                                  dataset="all",
                                                  shuffle=False,
                                                  save_path=".",
                                                  save_predictions=True
                                                 )

In [11]:
torch_sim_metrics["test"]

{'ML': {'MSE': {'x-velocity': 1791.7400313875885,
   'y-velocity': 1698.7715926507387,
   'pressure': 64861130.19502665,
   'turbulent_viscosity': 9.796630593431895e-06},
  'MSE_surfacic': {'pressure': 9.796630593431895e-06},
  'MAPE': {'x-velocity': 1094019235195831.4,
   'y-velocity': 344373520588708.94,
   'pressure': 133.0833960904364,
   'turbulent_viscosity': 30288872610.038124},
  'MAPE_surfacic': {'pressure': 30288872610.038124}},
 'Physics': {'spearman_correlation_drag': 0.04170516746618518,
  'spearman_correlation_lift': 0.010670081092616303,
  'mean_relative_drag': 5.4570039360906755,
  'std_relative_drag': 5.920070666670164,
  'mean_relative_lift': 2.5534989646928663,
  'std_relative_lift': 8.534126404295673},
 'IndRed': {}}